In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import glob
import seaborn as sns
 
from nback_utils import read_nback_results, summarize_nback

pd.options.display.max_rows = 100

# Run today

In [4]:
today = read_nback_results('/Users/olgabot/Downloads/N-back.data.2022-05-08--01-00.txt')
today_summary = summarize_nback(today).to_frame().T
today_summary.to_clipboard(index=False)

/Users/olgabot/opt/miniconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
